A basic momentum strategy

In [2]:
import pandas as pd

from quantresearch.interface.binance import get_binance_close

# Parameters of our strategy
univ = ['BTCUSDT','ETHUSDT','ADAUSDT','BNBUSDT','XRPUSDT','DOTUSDT','MATICUSDT']
data_period = 4 # in hours
train_start = '2020-01-01'
train_end = '2022-12-31'
test_start = '2023-01-01'
test_end = '2023-12-31'

freq = f'{data_period}h'
data_train = {x: get_binance_close(x, freq, start_ts=train_start, end_ts=train_end) for x in univ}
data_train = pd.DataFrame.from_dict(data_train, orient='columns')
data_train = data_train.reindex(pd.date_range(data_train.index[0], data_train.index[-1], freq=freq))
data_train

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2019-12-31 19:00:00,7230.71,130.18,0.03308,13.8159,0.19406,NaN,NaN
2019-12-31 23:00:00,7205.50,130.52,0.03320,13.7648,0.19518,NaN,NaN
2020-01-01 03:00:00,7195.80,130.84,0.03321,13.7162,0.19358,NaN,NaN
2020-01-01 07:00:00,7233.02,131.84,0.03357,13.7958,0.19428,NaN,NaN
2020-01-01 11:00:00,7223.72,131.98,0.03361,13.7270,0.19474,NaN,NaN
...,...,...,...,...,...,...,...
2022-12-30 03:00:00,16493.53,1190.67,0.24168,243.3964,NaN,4.24,0.757
2022-12-30 07:00:00,16553.99,1194.97,0.24332,245.0045,NaN,4.27,0.756
2022-12-30 11:00:00,16526.61,1194.98,0.24423,244.4105,NaN,4.33,0.758
2022-12-30 15:00:00,16609.35,1199.90,0.24531,245.5692,NaN,4.33,0.760


In [3]:
ret = data_train.pct_change(fill_method=None)
ret

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2019-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-31 23:00:00,-0.003487,0.002612,0.003628,-0.003699,0.005771,NaN,NaN
2020-01-01 03:00:00,-0.001346,0.002452,0.000301,-0.003531,-0.008198,NaN,NaN
2020-01-01 07:00:00,0.005172,0.007643,0.010840,0.005803,0.003616,NaN,NaN
2020-01-01 11:00:00,-0.001286,0.001062,0.001192,-0.004987,0.002368,NaN,NaN
...,...,...,...,...,...,...,...
2022-12-30 03:00:00,0.000939,0.002155,0.006078,-0.000619,NaN,0.000000,0.003979
2022-12-30 07:00:00,0.003666,0.003611,0.006786,0.006607,NaN,0.007075,-0.001321
2022-12-30 11:00:00,-0.001654,0.000008,0.003740,-0.002424,NaN,0.014052,0.002646
2022-12-30 15:00:00,0.005006,0.004117,0.004422,0.004741,NaN,0.000000,0.002639


In [4]:
from quantresearch.trading.algorithms.momentum import long_short

# For each of the following multiples of data_period, we make a momentum strategy and calculate the strategy return:
horizons = [1, 2, 3, 4, 5, 6]
strats_returns = {}
for h in horizons:
    # Calculate the portfolio weights for the strategy
    weights = long_short(ret, h)
    # Calculate the return of the strategy for each period
    strats_returns[h] = (weights.shift()*ret).sum(axis=1)
strats_returns = pd.DataFrame(strats_returns)
strats_returns

,1,2,3,4,5,6
2019-12-31 19:00:00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-12-31 23:00:00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-01 03:00:00,-1.281039e-03,-0.001281,-0.001281,-0.001281,-0.001281,-0.001281
2020-01-01 07:00:00,2.181741e-03,0.001558,0.001558,0.001558,0.001558,0.001558
2020-01-01 11:00:00,-7.816796e-07,0.000616,0.002451,0.002451,0.002451,0.002451
...,...,...,...,...,...,...
2022-12-30 03:00:00,-1.840816e-03,-0.001533,-0.001166,-0.001841,-0.000857,-0.002014
2022-12-30 07:00:00,-1.352729e-03,0.000792,0.000812,0.001511,0.000798,0.000798
2022-12-30 11:00:00,3.747462e-03,0.002786,-0.004049,-0.002178,-0.002350,-0.004233
2022-12-30 15:00:00,-1.496439e-03,-0.000796,-0.001362,-0.000865,-0.000835,-0.000865


In [5]:
from quantresearch.trading.evaluation.metric import sharpe_ratio

# Calculate the annualized Sharpe of each strategy
sr = sharpe_ratio(strats_returns, data_period)
sr

1   -0.989473
2   -0.238533
3    0.259369
4    1.071280
5    1.490998
6    0.899934
dtype: float64

In [6]:
# Since 5*4=20 hours gained the highest sharpe ratio, we pick this value as the trained parameter of our strategy
horizon_trained = 20

# Then, we test this strategy against the test data
data_test = {x: get_binance_close(x, freq, start_ts=test_start, end_ts=test_end) for x in univ}
data_test = pd.DataFrame.from_dict(data_test, orient='columns')
data_test = data_test.reindex(pd.date_range(data_test.index[0], data_test.index[-1], freq=freq))
data_test

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2022-12-31 19:00:00,16521.36,1194.19,0.24522,245.1529,NaN,4.290,0.7530
2022-12-31 23:00:00,16528.14,1195.32,0.24514,243.9536,NaN,4.310,0.7520
2023-01-01 03:00:00,16550.51,1196.17,0.24586,244.2534,NaN,4.330,0.7550
2023-01-01 07:00:00,16557.02,1196.59,0.24610,244.3091,NaN,4.330,0.7560
2023-01-01 11:00:00,16599.38,1201.98,0.24852,245.0832,NaN,4.380,0.7620
...,...,...,...,...,...,...,...
2023-12-30 03:00:00,41899.99,2285.90,0.59790,316.5000,0.6202,8.195,0.9574
2023-12-30 07:00:00,42394.18,2313.94,0.61010,318.8000,0.6288,8.385,0.9724
2023-12-30 11:00:00,42360.00,2299.00,0.60780,315.6000,0.6254,8.338,0.9677
2023-12-30 15:00:00,42138.01,2293.40,0.60140,316.8000,0.6219,8.367,0.9511


In [7]:
data_test_ret = data_test.pct_change(fill_method=None)
data_test_ret

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2022-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:00:00,0.000410,0.000946,-0.000326,-0.004892,NaN,0.004662,-0.001328
2023-01-01 03:00:00,0.001353,0.000711,0.002937,0.001229,NaN,0.004640,0.003989
2023-01-01 07:00:00,0.000393,0.000351,0.000976,0.000228,NaN,0.000000,0.001325
2023-01-01 11:00:00,0.002558,0.004504,0.009833,0.003169,NaN,0.011547,0.007937
...,...,...,...,...,...,...,...
2023-12-30 03:00:00,-0.000909,-0.004395,-0.007140,-0.005343,-0.003855,-0.015379,-0.011869
2023-12-30 07:00:00,0.011795,0.012267,0.020405,0.007267,0.013866,0.023185,0.015667
2023-12-30 11:00:00,-0.000806,-0.006457,-0.003770,-0.010038,-0.005407,-0.005605,-0.004833
2023-12-30 15:00:00,-0.005241,-0.002436,-0.010530,0.003802,-0.005596,0.003478,-0.017154


In [8]:
# Apply the trained strategy to the test data
weights = long_short(data_test_ret, horizon_trained)

# Calculate the return of the strategy over each test period
strat_ret = (weights.shift()*data_test_ret).sum(axis=1)
strat_ret

2022-12-31 19:00:00    0.000000
2022-12-31 23:00:00    0.000000
2023-01-01 03:00:00    0.000313
2023-01-01 07:00:00    0.000131
2023-01-01 11:00:00    0.003304
                         ...   
2023-12-30 03:00:00    0.003509
2023-12-30 07:00:00   -0.003829
2023-12-30 11:00:00   -0.001690
2023-12-30 15:00:00    0.004224
2023-12-30 19:00:00    0.005529
Freq: 4h, Length: 2185, dtype: float64

In [10]:
sr = sharpe_ratio(strat_ret, data_period)
sr

np.float64(0.756299226388108)